In [1]:
print(2)

2


In [24]:
import torch
from transformer import Transformer

START_TOKEN = '<START>'
PADDING_TOKEN = '<PADDING>'
END_TOKEN = '<END>'

trg_vocab = list(set(' abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZабвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ1234567890.,#!@"$;:^|\'-=+_—?<>'))
src_vocab = list(set(' abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZабвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ1234567890.,#!@"$;:^|\'-=+_—?<>'))

trg_vocab.append(START_TOKEN)
trg_vocab.append(END_TOKEN)
trg_vocab.append(PADDING_TOKEN)

src_vocab.append(START_TOKEN)
src_vocab.append(END_TOKEN)
src_vocab.append(PADDING_TOKEN)

index_to_trg = {k:v for k,v in enumerate(trg_vocab)}
trg_to_index = {v:k for k,v in enumerate(trg_vocab)}
index_to_src = {k:v for k,v in enumerate(src_vocab)}
src_to_index = {v:k for k,v in enumerate(src_vocab)}

d_model = 512
batch_size = 50
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.2
num_layers = 1
max_sequence_length = 200
trg_vocab_size = len(trg_vocab)

ru_eng_translator = Transformer(d_model, 
                            ffn_hidden,
                            num_heads, 
                            drop_prob, 
                            num_layers, 
                            max_sequence_length,
                            trg_vocab_size,
                            src_to_index,
                            trg_to_index,
                            index_to_trg,
                            START_TOKEN, 
                            END_TOKEN, 
                            PADDING_TOKEN)

eng_ru_translator = Transformer(d_model, 
                            ffn_hidden,
                            num_heads, 
                            drop_prob, 
                            num_layers, 
                            max_sequence_length,
                            trg_vocab_size,
                            src_to_index,
                            trg_to_index,
                            index_to_trg,
                            START_TOKEN, 
                            END_TOKEN, 
                            PADDING_TOKEN)

ru_eng_translator.load_state_dict(torch.load('models/ru_eng_trans.pth'))
eng_ru_translator.load_state_dict(torch.load('models/eng_ru_trans.pth'))

<All keys matched successfully>

In [25]:
def get_device():
    return torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

device = get_device()

In [26]:
ru_eng_translator.to(device)
eng_ru_translator.to(device)

ru_eng_translator.eval()
eng_ru_translator.eval()

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(152, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (ln): LayerNorm((512,), eps=512, elementwise_affine=True)
        )
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.2, inplace=False)
        (ffn): FeedFoward(
          (net): Sequential(
            (0): Linear(in_features=512, out_features=2048, bias=True)
            (1): ReLU()
            (2): Linear(in_features=2048, out_features=512, bias=True)
            (3): Dropout(p=0.2, inplace=False)
          )
        )
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout2): Dropout(p=0.2, inplace=F

In [27]:
ru_eng_translator.translate("Привет, как дела?")

'g$$.XЫЭ$!.M$d6ЫgбГк$кT!Md!кл$'

In [28]:
eng_ru_translator.translate("Hi, how are you?")

'ЙR<ULТJR<ЫbVЫbНVKТLТnШR'

In [19]:
model1 = torch.load("models/eng_ru_trans.pt")
model2 = torch.load("models/ru_eng_trans.pt")

RuntimeError: PytorchStreamReader failed locating file constants.pkl: file not found

In [20]:
model1.eval()

AttributeError: 'collections.OrderedDict' object has no attribute 'eval'

In [16]:
from transformer import *

In [17]:
def translate(model, src_sentence):
        
        model.eval()
        max_sequence_length = model.max_sequence_length
        device = get_device()
        src_sentence = (src_sentence,)
        trg_sentence = ("",)
        
        for word_counter in range(max_sequence_length):

            encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask=create_masks(src_sentence, trg_sentence)
            predictions = model(src_sentence,
                                    trg_sentence,
                                    encoder_self_attention_mask.to(device), 
                                    decoder_self_attention_mask.to(device), 
                                    decoder_cross_attention_mask.to(device),
                                    enc_start_token=False,
                                    enc_end_token=False,
                                    dec_start_token=True,
                                    dec_end_token=False)
            
            next_token_prob_distribution = predictions[0][word_counter]
            next_token_index = torch.argmax(next_token_prob_distribution).item()
            next_token = model.index_to_trg[next_token_index]
            
            if next_token == model.END_TOKEN:
                break

            trg_sentence = (trg_sentence[0] + next_token, )
        return trg_sentence[0]

In [18]:
translate(model1, "wad")

AttributeError: 'collections.OrderedDict' object has no attribute 'eval'